In [ ]:
import benepar
import os
from IPython.core.debugger import set_trace
import numpy as np

parser = benepar.Parser("benepar_en2_large")

save_dir = "features"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [ ]:
hp_text = open("chapter9.txt","r").readlines()
all_text = []
para = ""
for i in range(len(hp_text)):
    hp_text[i] = hp_text[i].strip()
    if hp_text[i]!="+":
        para += hp_text[i] + " "
    else:
        all_text.append(para)
        para = ""

In [ ]:
all_subtrees = []
for i in range(len(hp_text)):
    all_subtrees.append([])
    
all_lexical_prods = []
    
ptr = 0
for para in all_text:
    trees = parser.parse_sents(para)
    for tree in trees:
        productions = tree.productions()
        cur_prods = []
        len_remaining = len(hp_text[ptr])
        for prod in productions:
            cur_prods.append(prod)
            if prod.is_lexical():
                if prod.rhs()[0] == "``" or prod.rhs()[0] == "''" or prod.rhs()[0] == "-LRB-" or prod.rhs()[0] == "-RRB-":
                        len_remaining -= 1
                else:
                    len_remaining -= len(prod.rhs()[0])
                all_lexical_prods.append(prod)
            if len_remaining == 0:
                all_subtrees[ptr] = cur_prods.copy()
                ptr += 1
                if hp_text[ptr] == "+":
                    ptr += 1
                if ptr<len(hp_text) and hp_text[ptr] == "+":
                    ptr += 1
                if ptr<len(hp_text):
                    len_remaining = len(hp_text[ptr])

In [ ]:
import pickle
with open("all_subtrees_nodecnt.pkl","wb") as f:
    pickle.dump(all_subtrees,f)

In [ ]:
with open("all_subtrees_nodecnt.pkl","rb") as f:
    bck = pickle.load(f)
all_end_subtrees = []
for i in range(len(hp_text)):
    all_end_subtrees.append([])

node_cnts = []
for i in range(len(hp_text)):
    node_cnts.append(0)

for i in range(len(hp_text)):
    j = len(all_subtrees[i]) - 1
    prev = []
    while(j>=0):
        cur = bck[i][j]
        rhs = cur.rhs()
        if cur.is_lexical() and len(cur._rhs)>0:
            all_end_subtrees[i] = [all_subtrees[i][j]] + all_end_subtrees[i]
            cur._rhs = ()
            j -= 1
            prev.append(cur.lhs())
            node_cnts[i] += 1
        else:
            checkrem = 0
            rem = 0
            for sym in rhs:
                if sym in prev:
                    rem += 1
                    prev.remove(sym)
                    checkrem = 1
                if len(prev) == 0:
                    break
            bck[i][j]._rhs = bck[i][j]._rhs[rem:]
            if checkrem == 1 and len(bck[i][j]._rhs) == 0:
                node_cnts[i] += 1
                all_end_subtrees[i] = all_subtrees[i][j:]
                prev.append(all_subtrees[i][j].lhs())
            j -= 1

In [ ]:
for i in range(len(all_end_subtrees)):
    print(hp_text[i])
    print(node_cnts[i])
    print(all_subtrees[i])
    print(all_end_subtrees[i])

In [ ]:
np.save(os.path.join(save_dir,"node_count"),node_cnts)